In [2]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, concatenate
from keras.layers import Input, Conv2DTranspose
from keras.models import Model
from ipykernel import kernelapp as app
from keras import optimizers
from keras.regularizers import l2
#from keras import backend as K

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization



Using TensorFlow backend.


In [3]:

import sys
import SimpleITK as sitk
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
%matplotlib inline 
import numpy as np
import os
import fnmatch
import shutil
from ipywidgets import interact, fixed
import pandas as pd
import warnings
from skimage import exposure
from skimage import data
from skimage.transform import rotate
import _pickle as cPickle
import random



def myshow(image_arr,slice_n):
    plt.imshow(image_arr[slice_n,:,:],cmap='gray')
    plt.show()

In [4]:
df=pd.read_pickle('e://Image Processing/Brats Project/im+masks.pkl')

def get_arrays(data, column=['X', 'Y']):
    
    x=np.array([i for i in data[column[0]]]) #array of images
    y=np.array([i for i in data[column[1]]])#array for labels
    
    return x,y 

PATH_images='e://Image Processing/Brats Project/data/images/'
PATH_masks='e://Image Processing/Brats Project/data/masks/'

In [5]:
new_cols=['X', 'Y_1', 'Y_2', 'Y_3']
df.columns=new_cols

In [6]:
#getting arrays from columns 

train, test = train_test_split(df, test_size=0.20)

x_train, y_train = get_arrays(train, column=['X', 'Y_1'])
x_test, y_test = get_arrays(test, column=['X', 'Y_1'])

In [ ]:
tf.cast(image, tf.float32)

In [15]:
# create the training datasets
dx_train = tf.data.Dataset.from_tensor_slices(tf.cast(x_train,tf.float32))

# apply a one-hot transformation to each label for use in the neural network
dy_train = tf.data.Dataset.from_tensor_slices(y_train).map(lambda z: tf.one_hot(z, 2))

# zip the x and y training data together and shuffle, batch etc.
train_dataset = tf.data.Dataset.zip((dx_train, dy_train)).shuffle(500).repeat().batch(30)

In [ ]:
train_dataset

In [16]:
# do the same operations for the validation set
dx_valid = tf.data.Dataset.from_tensor_slices(tf.cast(x_test,tf.float32))
dy_valid = tf.data.Dataset.from_tensor_slices(y_test).map(lambda z: tf.one_hot(z, 2))
valid_dataset = tf.data.Dataset.zip((dx_valid, dy_valid)).shuffle(500).repeat().batch(30)

In [ ]:
valid_dataset 

In [17]:
# create general iterator
iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
next_element = iterator.get_next()

In [18]:
# make datasets that we can initialize separately, but using the same structure via the common iterator
training_init_op = iterator.make_initializer(train_dataset)
validation_init_op = iterator.make_initializer(valid_dataset)

In [ ]:
train_dataset

In [19]:
def nn_model(in_data):
    bn = tf.layers.batch_normalization(in_data)
    fc1 = tf.layers.dense(bn, 50)
    fc2 = tf.layers.dense(fc1, 50)
    fc2 = tf.layers.dropout(fc2)
    fc3 = tf.layers.dense(fc2, 2)
    
    
    return fc3

In [20]:
# create the neural network model
logits = nn_model(next_element[0])

In [21]:
# add the optimizer and loss
loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits_v2(labels=next_element[1], logits=logits))
optimizer = tf.train.AdamOptimizer().minimize(loss)
# get accuracy
prediction = tf.argmax(logits, 1)
equality = tf.equal(prediction, tf.argmax(next_element[1], 1))
accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))
init_op = tf.global_variables_initializer()

In [22]:
# run the training
config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC'
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.90
#sess = tf.Session(config=config)
#K.set_session(sess)




epochs = 10
with tf.Session(config=config) as sess:
    sess.run(init_op)
    sess.run(training_init_op)
    for i in range(epochs):
        l, _, acc = sess.run([loss, optimizer, accuracy])
        
        print("Epoch: {}, loss: {:.3f}, training accuracy: {:.2f}%".format(i, l, acc * 100))
    # now setup the validation run
    valid_iters = 10
    # re-initialize the iterator, but this time with validation data
    sess.run(validation_init_op)
    avg_acc = 0
    for i in range(valid_iters):
        acc = sess.run([accuracy])
        avg_acc += acc[0]
    print("Average validation set accuracy over {} iterations is {:.2f}%".format(valid_iters,(avg_acc / valid_iters) * 100))

Epoch: 0, loss: 1906237.500, training accuracy: 54.96%
Epoch: 1, loss: 587205.750, training accuracy: 52.54%
Epoch: 2, loss: 268668.406, training accuracy: 54.43%
Epoch: 3, loss: 309623.219, training accuracy: 49.82%
Epoch: 4, loss: 283562.531, training accuracy: 55.28%
Epoch: 5, loss: 272216.500, training accuracy: 59.40%
Epoch: 6, loss: 386269.844, training accuracy: 58.86%
Epoch: 7, loss: 304664.156, training accuracy: 69.90%
Epoch: 8, loss: 393361.688, training accuracy: 64.79%
Epoch: 9, loss: 614823.750, training accuracy: 66.72%
Average validation set accuracy over 10 iterations is 68.33%


In [ ]:
def vox_preprocess(vox):
    vox_shape = vox.shape
    vox = np.reshape(vox, (-1, vox_shape[-1]))
    vox = scale(vox, axis=0)
    return np.reshape(vox, vox_shape)


In [ ]:
y_train.shape

In [ ]:
num_classees=2
a = vox_preprocess(y_train)


In [1]:
a.shape

NameError: name 'a' is not defined

In [ ]:
imgs_train = x_train.astype('float32') 
imgs_val = x_test.astype('float32')
imgs_train /= 255. 
imgs_val /= 255.
    # imgs_train_1 = imgs_train[:, :, :, 0]
    # imgs_train_1 = imgs_train_1[..., np.newaxis]
    # print(imgs_train_1.shape)
    
    # define data preparation
batch_size = 12
datagen = ImageDataGenerator(featurewise_center=True,
                                featurewise_std_normalization=True,
                                 rotation_range=20, # rescale=1./255,
                                 width_shift_range=0.1, height_shift_range=0.1,
                                 horizontal_flip=True, vertical_flip=True,                                 
                                 shear_range=0.2, fill_mode='nearest')
#==============================================================================
#     datagen = ImageDataGenerator(rotation_range=20, # rescale=1./255,
#                                  width_shift_range=0.1, height_shift_range=0.1,
#                                  horizontal_flip=True, vertical_flip=True,                                 
#                                  shear_range=0.2, fill_mode='nearest')
#==============================================================================
# fit parameters from data
datagen.fit(imgs_train)     
# datagen.fit(imgs_train_1)
train_generator = datagen.flow(imgs_train, Y, batch_size=batch_size)

In [ ]:
# this is the augmentation configuration we will use for testing:
val_datagen = ImageDataGenerator(featurewise_center=True,
                                     featurewise_std_normalization=True)
                                     # rescale=1./255)
# val_datagen = ImageDataGenerator(rescale=1./255)
    
# fit parameters from data
val_datagen.fit(imgs_val) 
# val_datagen.fit(imgs_train)   
val_generator = val_datagen.flow(imgs_val, Y_test, batch_size=batch_size)
# datagen.standardize(imgs_val)

In [ ]:
y_tr_0=np.reshape(y_train[150], 120*120).astype('int32')
y_tr_0 = np_utils.to_categorical(y_tr_0, 2)
y_tr_0=y_tr_0[np.newaxis, ...]

In [ ]:
def make_labels(labels):
    new_labels=np.empty((labels.shape[0], 120*120, 2), dtype='int32')
    
    for i,y in enumerate(labels):
        im=np.reshape(y, 120*120).astype('int32')
        im=np_utils.to_categorical(im, 2)
        im=im[np.newaxis, ...]
        new_labels[i]=im
    return new_labels

In [ ]:
Y=make_labels(y_train)
Y_test=make_labels(y_test)

In [ ]:
generator = train_datagen.flow_from_directory(y_train, batch_size=32)
#just call

y_true_labels = generator.classes

In [ ]:
import keras.backend as K
def f1_score(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1.) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1.)

def f1_loss(y_true, y_pred):
    return -f1_score(y_true, y_pred)

In [ ]:
#epochs = 3

#Adam = keras.optimizers.Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon= 0.001,decay=0.00, amsgrad=False)
#my_model.compile(optimizer=Adam, loss ='categorical_crossentropy', metrics=['accuracy'])
my_model = cnnBRATsInit_unet()
my_model.fit_generator(train_generator, steps_per_epoch=(imgs_train.shape[0] // batch_size) + 1, epochs=20,                        
                                  verbose=1,
                                  validation_data=val_generator,
                                  validation_steps=(imgs_val.shape[0] // batch_size) + 1)

score = my_model.evaluate(val_generator, batch_size=32)
my_model.metrics_names , score

In [ ]:
vgg_model = keras.applications.vgg16.VGG16(include_top=False, weights='imagenet')


In [ ]:
input = Input(shape=(120,120,3),name = 'image_input')

#Use the generated model 
input_nor = BatchNormalization()(input)
output_vgg16_conv = vgg_model(input_nor)

#Add the fully-connected layers 
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(5, activation='softmax', name='predictions')(x)

#Create your own model 
my_model = Model(input=input, output=x)

#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
#my_model.summary()